In [1]:
import cv2
import numpy as np
from matplotlib import pyplot as plt
from models.detector import face_detector
from models.detector.iris_detector import IrisDetector
import sys
import warnings
warnings.filterwarnings("ignore")

C:\Users\jason\anaconda3\envs\yolov4\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\jason\anaconda3\envs\yolov4\lib\site-packages\tensorflow\python\framework\dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\jason\anaconda3\envs\yolov4\lib\site-packages\tensorflow\python\framework\dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\jason\anaconda3\envs\yolov4\lib\site-packages\tensorflow\pyth

In [2]:
fd = face_detector.FaceAlignmentDetector(
    lmd_weights_path="./models/detector/FAN/2DFAN-4_keras.h5",
     fd_type="s3fd"
    # 2DFAN-4_keras.h5, 2DFAN-1_keras.h5
)

Instructions for updating:
Colocations handled automatically by placer.
loading S3FD model............
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, None, None, 3 0                                            
__________________________________________________________________________________________________
conv1_1 (Conv2D)                (None, None, None, 6 1792        input_1[0][0]                    
__________________________________________________________________________________________________
conv1_2 (Conv2D)                (None, None, None, 6 36928       conv1_1[0][0]                    
__________________________________________________________________________________________________
max_pooling2d_1 (MaxPooling2D)  (None, None, None, 6 0           conv1_2[0][0]                    
______

## warm up

In [3]:
def resize_image(im, max_size=768):
    if np.max(im.shape) > max_size:
        ratio = max_size / np.max(im.shape)
        print(f"Resize image to ({str(int(im.shape[1]*ratio))}, {str(int(im.shape[0]*ratio))}).")
        return cv2.resize(im, (0,0), fx=ratio, fy=ratio)
    return im


# Test images are obtained on https://www.pexels.com/
im = cv2.imread(r"C:\Users\jason\Facial-Recognition-FaceNet-master\database\id\eric.jpg")[..., ::-1]
im = resize_image(im) # Resize image to prevent GPU OOM.
bboxes, landmarks = fd.detect_face(im, with_landmarks=True)

Resize image to (768, 432).
Building FAN for landmarks detection...
loading FAN model............
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input0 (InputLayer)             (None, 3, 256, 256)  0                                            
__________________________________________________________________________________________________
conv10.23005182903749222_pad (Z (None, 3, 262, 262)  0           input0[0][0]                     
__________________________________________________________________________________________________
conv10.23005182903749222 (Conv2 (None, 64, 128, 128) 9472        conv10.23005182903749222_pad[0][0
__________________________________________________________________________________________________
bn10.20053864458101422 (BatchNo (None, 64, 128, 128) 256         conv10.23005182903749222[0][0]   
___________

                                                                 m3.b2_3.conv30.7564490440701334[0
__________________________________________________________________________________________________
m3.b2_30.07988097514970227 (Add (None, 256, 16, 16)  0           m3.b2_30.8588218276337379[0][0]  
                                                                 m30.09693246072214201[0][0]      
__________________________________________________________________________________________________
m30.11832737839392615 (ZeroPadd (None, 256, 16, 16)  0           m3.b2_30.07988097514970227[0][0] 
__________________________________________________________________________________________________
m30.40266960279041997_pad (Zero (None, 256, 16, 16)  0           m30.11832737839392615[0][0]      
__________________________________________________________________________________________________
m30.40266960279041997 (AverageP (None, 256, 8, 8)    0           m30.40266960279041997_pad[0][0]  
__________

## landmarks

In [4]:
webcam = cv2.VideoCapture(0)
if not webcam.isOpened():
    sys.exit("Could not open webcam, please open the camera!")
count = 0
while True:
    count= count+1
    # read frame from webcam 
    status, frame = webcam.read()     
    frame_size = frame.shape
    face, landmarks = fd.detect_face(frame, with_landmarks=True)

    number = len(face)
    
    if number!=0:
        
        print(f"detect {number} face(s)!")
        print(face)
        print(" ")
        print("landmarks:",landmarks)
        
        for i in range(number):
            
            for j in range(len(landmarks[i])): 
                
                x, y = landmarks[i][j]
                cv2.circle(frame, (int(y), int(x)), 3, (0,255,0), -1) 
      
    else:
        
        print("do not detect face(s))!")

    cv2.namedWindow("demo", cv2.WINDOW_NORMAL)
    cv2.imshow("demo", frame)

    # press "Q" to stop
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# release resources
webcam.release()
cv2.destroyAllWindows()    

detect 1 face(s)!
[array([259.6720962524414, 235.2655838727951, 447.10131072998047,
       376.1067246198654, array([0.99999857], dtype=float32)],
      dtype=object)]
 
landmarks: [array([[344., 227.],
       [366., 233.],
       [387., 238.],
       [405., 243.],
       [424., 254.],
       [439., 267.],
       [450., 280.],
       [455., 298.],
       [460., 322.],
       [450., 346.],
       [439., 356.],
       [426., 364.],
       [410., 372.],
       [387., 377.],
       [368., 380.],
       [347., 383.],
       [326., 380.],
       [318., 240.],
       [310., 251.],
       [305., 262.],
       [305., 272.],
       [308., 283.],
       [305., 325.],
       [300., 335.],
       [297., 343.],
       [297., 356.],
       [302., 364.],
       [331., 306.],
       [347., 309.],
       [360., 312.],
       [371., 312.],
       [381., 301.],
       [384., 306.],
       [384., 312.],
       [381., 319.],
       [379., 325.],
       [337., 262.],
       [334., 269.],
       [334., 277.],

detect 1 face(s)!
[array([259.0219078063965, 234.20068204402924, 457.90920639038086,
       377.33967435359955, array([0.9999943], dtype=float32)],
      dtype=object)]
 
landmarks: [array([[343., 227.],
       [362., 230.],
       [384., 238.],
       [403., 244.],
       [423., 252.],
       [439., 265.],
       [450., 279.],
       [458., 295.],
       [461., 320.],
       [453., 345.],
       [442., 356.],
       [428., 366.],
       [412., 375.],
       [390., 377.],
       [371., 380.],
       [349., 383.],
       [327., 380.],
       [319., 241.],
       [311., 249.],
       [308., 263.],
       [308., 274.],
       [311., 282.],
       [308., 325.],
       [305., 334.],
       [300., 345.],
       [302., 356.],
       [308., 364.],
       [335., 306.],
       [349., 306.],
       [362., 309.],
       [373., 309.],
       [384., 298.],
       [387., 304.],
       [387., 309.],
       [384., 317.],
       [384., 323.],
       [338., 260.],
       [338., 268.],
       [335., 276.]

detect 1 face(s)!
[array([244.63327598571777, 232.7303466796875, 456.0707759857178,
       380.81890869140625, array([0.99996233], dtype=float32)],
      dtype=object)]
 
landmarks: [array([[334., 223.],
       [357., 226.],
       [377., 232.],
       [398., 238.],
       [418., 249.],
       [435., 261.],
       [444., 275.],
       [452., 293.],
       [458., 318.],
       [450., 344.],
       [438., 356.],
       [426., 368.],
       [406., 376.],
       [383., 382.],
       [363., 385.],
       [343., 388.],
       [320., 385.],
       [308., 238.],
       [300., 246.],
       [294., 261.],
       [297., 272.],
       [300., 281.],
       [297., 324.],
       [294., 336.],
       [291., 347.],
       [291., 359.],
       [300., 368.],
       [326., 307.],
       [340., 304.],
       [354., 307.],
       [366., 307.],
       [377., 295.],
       [380., 301.],
       [380., 307.],
       [377., 316.],
       [377., 321.],
       [328., 258.],
       [326., 264.],
       [326., 275.]

detect 1 face(s)!
[array([130.9027862548828, 206.19747352600098, 409.96604919433594,
       398.6873722076416, array([1.], dtype=float32)], dtype=object)]
 
landmarks: [array([[249., 198.],
       [279., 201.],
       [309., 209.],
       [336., 217.],
       [362., 224.],
       [381., 243.],
       [396., 258.],
       [407., 277.],
       [415., 311.],
       [403., 341.],
       [388., 360.],
       [373., 375.],
       [351., 390.],
       [320., 398.],
       [294., 402.],
       [264., 409.],
       [234., 413.],
       [215., 217.],
       [203., 228.],
       [196., 243.],
       [200., 258.],
       [203., 269.],
       [203., 330.],
       [200., 341.],
       [196., 356.],
       [196., 375.],
       [207., 386.],
       [241., 303.],
       [260., 300.],
       [279., 300.],
       [294., 300.],
       [309., 285.],
       [309., 292.],
       [313., 300.],
       [309., 311.],
       [305., 318.],
       [241., 239.],
       [237., 247.],
       [237., 262.],
       [245.

detect 1 face(s)!
[array([135.20404958724976, 204.13826751708984, 436.739022731781,
       413.2307357788086, array([1.], dtype=float32)], dtype=object)]
 
landmarks: [array([[251., 196.],
       [284., 200.],
       [312., 204.],
       [341., 212.],
       [369., 220.],
       [394., 236.],
       [410., 257.],
       [427., 277.],
       [439., 310.],
       [431., 343.],
       [410., 363.],
       [394., 380.],
       [365., 396.],
       [333., 404.],
       [304., 412.],
       [271., 416.],
       [243., 420.],
       [214., 212.],
       [198., 224.],
       [194., 241.],
       [198., 257.],
       [202., 269.],
       [198., 335.],
       [194., 347.],
       [190., 363.],
       [194., 380.],
       [210., 396.],
       [251., 302.],
       [275., 302.],
       [296., 302.],
       [316., 302.],
       [329., 286.],
       [329., 290.],
       [333., 302.],
       [329., 314.],
       [329., 322.],
       [247., 232.],
       [247., 245.],
       [247., 257.],
       [255.,

...

detect 1 face(s)!
[array([245.29978942871094, 191.6105556488037, 473.1382751464844,
       357.34450340270996, array([0.9999981], dtype=float32)],
      dtype=object)]
 
landmarks: [array([[342., 193.],
       [367., 187.],
       [393., 190.],
       [415., 196.],
       [437., 202.],
       [453., 212.],
       [465., 215.],
       [475., 221.],
       [484., 240.],
       [478., 266.],
       [465., 294.],
       [450., 319.],
       [431., 338.],
       [405., 351.],
       [383., 357.],
       [358., 360.],
       [330., 363.],
       [307., 196.],
       [295., 196.],
       [292., 199.],
       [292., 206.],
       [298., 212.],
       [298., 250.],
       [295., 259.],
       [292., 272.],
       [295., 288.],
       [307., 303.],
       [336., 225.],
       [355., 218.],
       [371., 212.],
       [383., 212.],
       [393., 215.],
       [396., 215.],
       [399., 221.],
       [396., 228.],
       [396., 237.],
       [330., 199.],
       [326., 202.],
       [330., 209.],

detect 1 face(s)!
[array([239.50435209274292, 215.50817775726318, 477.1050753593445,
       383.64401149749756, array([0.99999905], dtype=float32)],
      dtype=object)]
 
landmarks: [array([[337., 209.],
       [363., 209.],
       [389., 216.],
       [409., 222.],
       [431., 232.],
       [451., 245.],
       [461., 258.],
       [470., 274.],
       [477., 303.],
       [470., 329.],
       [454., 349.],
       [438., 362.],
       [415., 372.],
       [386., 381.],
       [363., 385.],
       [340., 388.],
       [314., 391.],
       [305., 216.],
       [295., 225.],
       [292., 238.],
       [292., 251.],
       [295., 261.],
       [295., 310.],
       [288., 323.],
       [285., 336.],
       [285., 352.],
       [295., 362.],
       [331., 290.],
       [350., 290.],
       [366., 287.],
       [379., 287.],
       [389., 277.],
       [392., 284.],
       [392., 290.],
       [392., 300.],
       [389., 307.],
       [331., 235.],
       [331., 245.],
       [327., 255.

detect 1 face(s)!
[array([229.12050318717957, 217.95390033721924, 470.71547389030457,
       385.25004291534424, array([0.99999905], dtype=float32)],
      dtype=object)]
 
landmarks: [array([[328., 210.],
       [358., 214.],
       [381., 220.],
       [404., 223.],
       [427., 233.],
       [443., 250.],
       [456., 263.],
       [466., 279.],
       [472., 309.],
       [463., 335.],
       [450., 351.],
       [436., 364.],
       [413., 374.],
       [387., 381.],
       [364., 387.],
       [341., 391.],
       [315., 391.],
       [299., 223.],
       [289., 233.],
       [286., 246.],
       [286., 260.],
       [289., 273.],
       [289., 322.],
       [286., 332.],
       [282., 345.],
       [286., 361.],
       [292., 371.],
       [325., 299.],
       [341., 299.],
       [358., 299.],
       [371., 299.],
       [384., 286.],
       [384., 289.],
       [387., 299.],
       [384., 309.],
       [381., 315.],
       [325., 243.],
       [322., 253.],
       [322., 263

detect 1 face(s)!
[array([231.14520454406738, 218.67489051818848, 470.9524555206299,
       385.06919288635254, array([0.99999905], dtype=float32)],
      dtype=object)]
 
landmarks: [array([[329., 212.],
       [355., 212.],
       [382., 218.],
       [404., 225.],
       [427., 234.],
       [443., 247.],
       [456., 260.],
       [466., 280.],
       [476., 306.],
       [463., 335.],
       [450., 351.],
       [434., 364.],
       [411., 374.],
       [385., 384.],
       [362., 387.],
       [339., 390.],
       [313., 394.],
       [300., 221.],
       [290., 231.],
       [287., 244.],
       [287., 257.],
       [290., 270.],
       [290., 319.],
       [287., 332.],
       [284., 345.],
       [284., 361.],
       [294., 371.],
       [323., 296.],
       [342., 296.],
       [359., 296.],
       [372., 296.],
       [382., 283.],
       [385., 290.],
       [388., 299.],
       [385., 309.],
       [382., 316.],
       [323., 241.],
       [323., 251.],
       [323., 260.

detect 1 face(s)!
[array([232.21121273189783, 213.1863832473755, 471.62417659908533,
       381.2498598098755, array([0.99999905], dtype=float32)],
      dtype=object)]
 
landmarks: [array([[330., 204.],
       [357., 207.],
       [379., 213.],
       [402., 220.],
       [425., 230.],
       [445., 246.],
       [458., 259.],
       [467., 279.],
       [474., 308.],
       [464., 334.],
       [451., 347.],
       [435., 360.],
       [415., 370.],
       [389., 376.],
       [366., 380.],
       [344., 383.],
       [317., 383.],
       [304., 220.],
       [295., 233.],
       [291., 246.],
       [295., 259.],
       [298., 272.],
       [295., 321.],
       [291., 334.],
       [288., 347.],
       [288., 360.],
       [295., 370.],
       [327., 298.],
       [347., 301.],
       [363., 301.],
       [376., 301.],
       [386., 288.],
       [389., 292.],
       [389., 301.],
       [386., 311.],
       [386., 318.],
       [327., 243.],
       [327., 252.],
       [327., 262.]

In [ ]:
print(a[1])
print(c[0])

## face

In [ ]:
shrink_frame_scale=0.25
recover_size = lambda x: x*(int(1/shrink_frame_scale))

webcam = cv2.VideoCapture(0)
if not webcam.isOpened():
    sys.exit("Could not open webcam, please open the camera!")

while True:
    
    # read frame from webcam 
    status, frame = webcam.read()     
    frame_size = frame.shape
    
    small_frame = cv2.resize(frame, (0, 0), fx=shrink_frame_scale, fy=shrink_frame_scale)

    h, w, _ = frame.shape
    
    bboxes = fd.detect_face(small_frame, with_landmarks=False)
    
    number0 = len(bboxes)
    number = len(bboxes[0])
    
    print(bboxes)
    
    if number!=0:
   
        for i in range(number0):
            
             # Display detected face
            x0, y0, x1, y1, score = bboxes[i] 
            x0, y0, x1, y1 = map(int, [x0, y0, x1, y1])

            crop_face = small_frame[x0:x1, y0:y1, :]

            x0, y0, x1, y1 = map(recover_size,  [x0, y0, x1, y1])
            print(x0,y0,x1,y1)
            cv2.rectangle(frame, (y0,x0), (y1,x1), (0,255,0), 2)
  
    else:
        print("No face detected.")
        
        
    cv2.namedWindow("demo", cv2.WINDOW_NORMAL)
    cv2.imshow("demo", frame)
    
    # press "Q" to stop
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break


# release resources
webcam.release()
cv2.destroyAllWindows()    

## pupil

In [ ]:
idet = iris_detector.IrisDetector()
idet.set_detector(fd) # fd = face_detector.FaceAlignmentDetector()

In [ ]:
shrink_frame_scale= 0.25
recover_scale = int(1/shrink_frame_scale)
recover_frame = lambda x: x*(recover_scale)

webcam = cv2.VideoCapture(0)
if not webcam.isOpened():
    sys.exit("Could not open webcam, please open the camera!")

while True:
    
    # read frame from webcam 
    status, frame = webcam.read()     
    frame_size = frame.shape
    
    small_frame = cv2.resize(frame, (0, 0), fx=shrink_frame_scale, fy=shrink_frame_scale)

    h, w, _ = frame.shape
    

    eye_lms = idet.detect_iris(small_frame)
    left = eye_lms[0][0,...]
    right = eye_lms[0][1,...]
    
    print(left)

    for i in range(18):
        for k in range(2):
            left[i,k] = recover_scale*left[i,k]
            right[i,k] = recover_scale*right[i,k]
    print(left)
    frame = idet.draw_pupil(frame, left) # draw left eye
    frame = idet.draw_pupil(frame, right) # draw right eye

    cv2.namedWindow("demo", cv2.WINDOW_NORMAL)
    cv2.imshow("demo", frame)

        # press "Q" to stop
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

        
        
        
# release resources
webcam.release()
cv2.destroyAllWindows()    